In [1]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, AutoModel


In [2]:
import os
import json
import random
from collections import defaultdict

# Path to the train folder
train_folder = "/home/mh/Downloads/Data-part1&2-v3/train"

# Intent mapping based on the provided data
intent_mapping = {
    10: "open_account_free",
    11: "open_account_current",
    12: "open_account_deposit",
    20: "loan_free",
    21: "loan_interest",
    30: "card2card",
    31: "paya",
    32: "convert_cheque",
    40: "receipt_payment",
    41: "installment_payment",
    50: "turnover_bill",
    51: "balance_bill",
    60: "submit_cheque",
    61: "recieve_cheque",
    70: "change_password",
    71: "duplicate_card",
    72: "close_card",
    80: "delegate_account",
    81: "currency_request",
    90: "software_problem",
    91: "signin_problem"
}

# Function to group files by intent
def group_files_by_intent(folder_path):
    """
    Group JSON files in the folder by their intent_id.
    """
    grouped_data = defaultdict(list)

    for file_name in os.listdir(folder_path):
        if file_name.endswith(".json"):
            file_path = os.path.join(folder_path, file_name)
            with open(file_path, "r", encoding="utf-8") as f:
                data = json.load(f)
                intent_id = data.get("intent_id")
                if intent_id is not None:
                    grouped_data[intent_id].append(data)

    return grouped_data

# Function to sample data from each intent
def sample_from_intents(grouped_data, num_samples=1):
    """
    Sample a specified number of examples from each intent.
    """
    sampled_data = {}

    for intent_id, examples in grouped_data.items():
        sampled_data[intent_id] = random.sample(examples, min(num_samples, len(examples)))

    return sampled_data

# Main script
if __name__ == "__main__":
    # Group files by intent
    grouped_data = group_files_by_intent(train_folder)

    # Sample data from each intent (e.g., 1 sample per intent)
    num_samples_per_intent = 1  # Change this to sample more examples per intent
    sampled_data = sample_from_intents(grouped_data, num_samples=num_samples_per_intent)

    # Print the sampled data
    print(f"Sampled Data from Train Folder ({num_samples_per_intent} sample(s) per intent):\n")
    for intent_id, samples in sampled_data.items():
        print(f"Intent {intent_id} ({intent_mapping.get(intent_id, 'Unknown')}):")
        for sample in samples:
            print(json.dumps(sample, ensure_ascii=False, indent=4))
        print("\n" + "-" * 50 + "\n")

Sampled Data from Train Folder (1 sample(s) per intent):

Intent 41 (installment_payment):
{
    "input_text": "وام 8083988 پلکانیه. این ماه قسطش 75000000 اس که باید پرداخت کنم. قسط 17 اشه",
    "intent_id": 41,
    "slots": [
        "o",
        "b-loan_id",
        "o",
        "o",
        "o",
        "o",
        "b-installment_amount",
        "o",
        "o",
        "o",
        "o",
        "o",
        "o",
        "b-installment_n",
        "o"
    ]
}

--------------------------------------------------

Intent 10 (open_account_free):
{
    "input_text": "میخام حساب قرض‌الحسنه باز کنی. حساب بنام دریا حمیدی باشه. کدملیش هم 4967930814 تاریخ 11 فروردین 34 هم بدنیا اومده. اسم پدرش محمود",
    "intent_id": 10,
    "slots": [
        "o",
        "o",
        "o",
        "o",
        "o",
        "o",
        "o",
        "b-fname",
        "b-lname",
        "o",
        "o",
        "o",
        "b-national_id",
        "o",
        "b-birth_date",
        "i-birth_date",
   

In [1]:
import torch
from transformers import AutoTokenizer, XLMRobertaForTokenClassification, AutoModelForSequenceClassification
from sklearn.preprocessing import LabelEncoder
from llama_cpp import Llama
import json
import time  # For adding delay

# Load slot labels
def load_slot_labels(slot_file_path):
    with open(slot_file_path, "r", encoding="utf-8") as f:
        slot_labels = [line.strip() for line in f.readlines()]
    return slot_labels

# Load intent labels
def load_intent_labels(intent_file_path):
    with open(intent_file_path, "r", encoding="utf-8") as f:
        intent_labels = [line.strip() for line in f.readlines()]
    return intent_labels

# Intent-to-slot mapping
intent_slot_mapping = {
    "open_account_free": [ "fname", "lname", "national_id", "father_name", "birth_date", "address", "starter_amount", "activate_ib", "issuance_card" ],
    "open_account_current": ["fname", "lname", "national_id", "father_name", "birth_date", "address", "starter_amount",  "activate_ib" ,"issuance_card",  "shared_cheque", "cheque_n" ,"support"],
    "card2card": ["transfer_amount", "fname", "lname",  "receiver_card" ,"transfer_datetime", "transfer_reason", "cvv2", "trans_pass"],
    "paya": ["transfer_amount", "fname", "lname", "transfer_datetime", "transfer_reason",  "receiver_iban", "receiver_bank" , "static_pass","trans_periodic"],
    "convert_cheque": [ "transfer_reason", "cfname", "clname", "cnational_id", "sayad_id", "cheque_date", "transfer_datetime" , "static_pass" ],
    "receipt_payment": ["bill_id", "payment_id", "phone_number", "post_code"],
    "installment_payment": ["installment_amount", "loan_id", "installment_n"],
    "turnover_bill": ["account_id", "start_datetime", "end_datetime",  "min_amount", "max_amount" ,"trans_n"],
    "balance_bill": ["balance_datetime"],
    "submit_cheque": ["sayad_id", "cheque_datetime", "cheque_amount", "cheque_reason", "cfname", "clname", "cnational_id"],
    "recieve_cheque": ["sayad_id", "cheque_amount", "cfname", "clname", "cnational_id"],
    "change_password": ["card_number", "current_pass", "new_pass"],
    "duplicate_card": ["card_number", "renew_reason"],
    "close_card": ["card_number", "blocking_reason", "current_pass"],
    "delegate_account": ["account_id","name", "start_datetime", "end_datetime" , "advocacy_reason" ,  "b-ncid" ],
    "currency_request": ["currency" , "amount", "country"  ], 
    "loan_free": ["loan_reason", "zfname", "zlname", "znational_id", "insurance_req", "loan_amount", "loan_duration"],
    "open_account_deposit" : [ "fname" , "lname" , "national_id", "father_name" , "birth_date", "address", "starter_amount", "benefit_rate",  "issuance_card", "deposit_duration"]

}

# Initialize LabelEncoders for slots and intents
slot_file_path = "/home/mh/Desktop/NLU-prj/Data-part1&2-v3/slot.txt"
intent_file_path = "/home/mh/Desktop/NLU-prj/Data-part1&2-v3/intent.txt"

slot_labels = load_slot_labels(slot_file_path)
intent_labels = load_intent_labels(intent_file_path)

slot_label_encoder = LabelEncoder()
slot_label_encoder.fit(slot_labels)

intent_label_encoder = LabelEncoder()
intent_label_encoder.fit(intent_labels)

# Load the slot filling model
slot_model_path = "/home/mh/Desktop/NLU-prj/Data-part1&2-v3/slot filling/best_model.pth"
slot_model = XLMRobertaForTokenClassification.from_pretrained(
    "/home/mh/Documents/NLU-exe/xlm_roberta/xlm-roberta-base",
    num_labels=len(slot_labels)
)
slot_model.load_state_dict(torch.load(slot_model_path, map_location=torch.device("cuda")), strict=False)

slot_tokenizer = AutoTokenizer.from_pretrained("/home/mh/Documents/NLU-exe/xlm_roberta/xlm-roberta-base")

# Load the intent detection model
intent_model_path = "/home/mh/Desktop/NLU-prj/Data-part1&2-v3/intent-model/best_intent_model_bot_challenge"
intent_tokenizer = AutoTokenizer.from_pretrained(intent_model_path)
intent_model = AutoModelForSequenceClassification.from_pretrained(intent_model_path)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
slot_model.to(device)
intent_model.to(device)

# Function for intent detection
# Function for intent detection
def predict_intent(text, tokenizer, model, intent_label_encoder):
    model.eval()
    encoding = tokenizer(text, padding=True, truncation=True, max_length=512, return_tensors="pt").to(device)
    with torch.no_grad():
        outputs = model(input_ids=encoding["input_ids"], attention_mask=encoding["attention_mask"])
    logits = outputs.logits
    probabilities = torch.softmax(logits, dim=1)  # Convert logits to probabilities
    predicted_class = torch.argmax(probabilities, dim=1).item()
    predicted_intent = intent_label_encoder.inverse_transform([predicted_class])[0]

    # Get the confidence score of the detected intent
    confidence = probabilities[0][predicted_class].item()

    # Get the top 3 intents with their probabilities
    top_3_indices = torch.topk(probabilities, 3, dim=1).indices.squeeze().tolist()
    top_3_probs = torch.topk(probabilities, 3, dim=1).values.squeeze().tolist()
    top_3_intents = intent_label_encoder.inverse_transform(top_3_indices)

    print(f"Detected Intent: {predicted_intent} (Confidence: {confidence:.4f})")
    print("Top 3 probable intents:")
    for intent, prob in zip(top_3_intents, top_3_probs):
        print(f"  {intent}: {prob:.4f}")

    return predicted_intent

# Load the LLaMA model
llama_model_path = "/home/mh/Desktop/AVA-Llama-3-V2.i1-Q6_K.gguf"
llm = Llama(model_path=llama_model_path, n_gpu_layers=2048)

# Function for slot filling
def predict_slots(model, tokenizer, text, slot_label_encoder, current_intent):
    """
    Predict slots from the input text and normalize BIO labels (e.g., B-fname → fname).
    Map detected slots to intent-specific slots based on the intent.
    """
    tokens = tokenizer(text.split(), is_split_into_words=True, truncation=True, padding="max_length", max_length=512, return_tensors="pt").to(device)
    word_ids = tokens.word_ids()

    with torch.no_grad():
        logits = model(input_ids=tokens["input_ids"], attention_mask=tokens["attention_mask"]).logits
        predictions = torch.argmax(logits, dim=-1).squeeze().cpu().numpy()

    aligned_predictions = []
    prev_word_idx = None
    for word_idx, prediction in zip(word_ids, predictions):
        if word_idx is None or word_idx == prev_word_idx:
            continue
        aligned_predictions.append(slot_label_encoder.inverse_transform([prediction])[0])
        prev_word_idx = word_idx

    words = text.split()
    slots = {}
    current_slot = None
    for word, slot in zip(words, aligned_predictions):
        # Normalize BIO labels (e.g., B-fname → fname, I-fname → fname)
        normalized_slot = slot.replace("b-", "").replace("i-", "")
        if normalized_slot != "o":
            if current_slot == normalized_slot:
                # Append to the current slot value (for multi-token slots)
                slots[normalized_slot] += f" {word}"
            else:
                # Start a new slot
                slots[normalized_slot] = word
                current_slot = normalized_slot

    # Map detected slots to intent-specific slots
    mapped_slots = {}
    intent_slots = intent_slot_mapping.get(current_intent, [])

    for slot_name, slot_value in slots.items():
        if slot_name in intent_slots:
            # Only add valid slots based on the intent
            mapped_slots[slot_name] = slot_value
        else:
            # Handle semantically similar slots (e.g., cheque_reason → transfer_reason)
            similar_slot = find_similar_slot(slot_name, intent_slots)
            if similar_slot:
                mapped_slots[similar_slot] = slot_value

    return mapped_slots

# Helper function to find a similar slot in the intent's slots
def find_similar_slot(detected_slot, intent_slots):
    """
    Find a similar slot in the intent's slots based on semantic similarity.
    For example, if the detected slot is 'cheque_reason' and the intent is 'card2card',
    map it to 'transfer_reason'.
    """
    # Define semantic groups for similar slots
    semantic_groups = {
        "reason": ["transfer_reason", "cheque_reason", "loan_reason", "blocking_reason", "renew_reason", "advocacy_reason" ],
        "amount": ["transfer_amount", "loan_amount", "installment_amount", "cheque_amount" ,"min_amount" ,"max_amount" ],
        "datetime": ["transfer_datetime", "cheque_date", "start_datetime", "end_datetime"],
        "card": ["card_number", "receiver_card"],
        "name": ["fname", "lname", "cfname", "clname", "zfname", "zlname"],
        "national_id" : ["znational_id" , "cnational_id" , "national_id"],
         "duration" : ["loan_duration" ,"deposit_duration"]
    }

    for group, similar_slots in semantic_groups.items():
        if detected_slot in similar_slots:
            # Find the first matching slot in the intent's slots
            for intent_slot in intent_slots:
                if intent_slot in similar_slots:
                    return intent_slot

    # If no similar slot is found, return None
    return None

# Function to generate a question for one unfilled slot using LLaMA
def generate_question_with_llm(intent, slot):
    prompt = f"""
 شما یک دستیار چت بات هوشمند بانکی هستید ! از شما میخواهم برای شکاف ها زیر با توجه به دامنه٫ سوال به فارسی تولید کنید و از کاربر بپرسید
   ( واژه نامه بعضی واژگان: sayad_id  −>  شناسه صیادی چک # benefit_rate −> نرخ سود بانکی # issuance_card -> درخواست صدور کارت شتاب #  activate_ib -> فعالسازی اینترنت بانک  # trans_n -> تعداد تراکنش های بانکی  #  znational_id -> شماره ملی فرد ضامن  # cnational_id  -> کد ملی گیرنده (حامل) چک  #  "shared_cheque"  -> نیازمند دسته چک اشتراکی# "support"  -> تعهد (ضمانت) مالی # receiver_iban −> شماره شبای فرد گیرنده)  
دامنه بحث  : {intent}
 شکاف : {slot}

یک سوال باید بر اساس دامنه و مختصر و مرتبط باشد و به فارسی و خطاب به کاربر برای دریافت اطلاعات پرسیده شود.
"""
    output = llm(prompt, max_tokens=90, temperature=0.75, top_p=0.5)
    return output["choices"][0]["text"].strip()

# Chatbot main function
def chatbot():
    print("سلام! من یک چت‌بات بانکی هستم. لطفاً سوال خود را بپرسید.")
    current_intent = None
    filled_slots = {}
    unfilled_slots = []
    last_asked_slot = None

    while True:
        user_message = input("شما: ")
        if user_message.lower() in ["خروج", "بای"]:
            print("چت‌بات: خداحافظ!")
            break

        # Step 1: Predict intent if not already set
        if current_intent is None:
            current_intent = predict_intent(user_message, intent_tokenizer, intent_model, intent_label_encoder)
            unfilled_slots = intent_slot_mapping.get(current_intent, [])
            print(f"چت‌بات: نیت شما شناسایی شد: {current_intent}")

        # Step 2: Predict slots
        slots = predict_slots(slot_model, slot_tokenizer, user_message, slot_label_encoder, current_intent)
        for slot_name, slot_value in slots.items():
            # Update filled_slots and remove from unfilled_slots
            if slot_name in intent_slot_mapping[current_intent]:  # Only add valid slots
                filled_slots[slot_name] = slot_value
                if slot_name in unfilled_slots:
                    unfilled_slots.remove(slot_name)

        # Print filled and unfilled slots
        print("چت‌بات: وضعیت فعلی:")
        print("پر شده:")
        print(json.dumps(filled_slots, ensure_ascii=False, indent=4))
        print("پر نشده:")
        print(json.dumps(unfilled_slots, ensure_ascii=False, indent=4))

        # Step 3: Ask about the next unfilled slot
        if unfilled_slots:
            next_slot = unfilled_slots[0]
            last_asked_slot = next_slot

            question = generate_question_with_llm(current_intent, next_slot)
            print(f"چت‌بات: {question}")
        else:
            # All slots are filled, return JSON and exit
            result = {"intent": current_intent, "slots": filled_slots}
            print("چت‌بات: تمام اطلاعات لازم پر شده است. نتیجه:")
            print(json.dumps(result, ensure_ascii=False, indent=4))
            break

# Run the chatbot
if __name__ == "__main__":
    chatbot()

Some weights of XLMRobertaForTokenClassification were not initialized from the model checkpoint at /home/mh/Documents/NLU-exe/xlm_roberta/xlm-roberta-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/tmp/ipykernel_31167/3108691314.py:62: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `t

سلام! من یک چت‌بات بانکی هستم. لطفاً سوال خود را بپرسید.
چت‌بات: خداحافظ!


gpu based 

In [ ]:
import torch
from transformers import AutoTokenizer, XLMRobertaForTokenClassification
from sklearn.preprocessing import LabelEncoder

# Load slot labels
def load_slot_labels(slot_file_path):
    with open(slot_file_path, "r", encoding="utf-8") as f:
        slot_labels = [line.strip() for line in f.readlines()]
    return slot_labels

# Initialize slot label encoder
slot_file_path = "/home/mh/Desktop/NLU-prj/Data-part1&2-v3/slot.txt"
slot_labels = load_slot_labels(slot_file_path)
slot_label_encoder = LabelEncoder()
slot_label_encoder.fit(slot_labels)

# Load the slot-filling model
slot_model_path = "/home/mh/Desktop/NLU-prj/Data-part1&2-v3/slot filling/best_model.pth"
slot_model = XLMRobertaForTokenClassification.from_pretrained(
    "/home/mh/Documents/NLU-exe/xlm_roberta/xlm-roberta-base",
    num_labels=len(slot_labels)
)
slot_model.load_state_dict(torch.load(slot_model_path, map_location=torch.device("cuda")), strict=False)
slot_model.eval()

# Load the tokenizer
slot_tokenizer = AutoTokenizer.from_pretrained("/home/mh/Documents/NLU-exe/xlm_roberta/xlm-roberta-base")

# Set device (CPU or GPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
slot_model.to(device)

# Function to predict slots
def predict_slots(model, tokenizer, text, slot_label_encoder):
    """
    Predict slots from the input text and normalize BIO labels (e.g., B-fname → fname).
    """
    tokens = tokenizer(text.split(), is_split_into_words=True, truncation=True, padding="max_length", max_length=256, return_tensors="pt").to(device)
    word_ids = tokens.word_ids()

    with torch.no_grad():
        logits = model(input_ids=tokens["input_ids"], attention_mask=tokens["attention_mask"]).logits
        predictions = torch.argmax(logits, dim=-1).squeeze().cpu().numpy()

    aligned_predictions = []
    prev_word_idx = None
    for word_idx, prediction in zip(word_ids, predictions):
        if word_idx is None or word_idx == prev_word_idx:
            continue
        aligned_predictions.append(slot_label_encoder.inverse_transform([prediction])[0])
        prev_word_idx = word_idx

    words = text.split()
    slots = {}
    current_slot = None
    for word, slot in zip(words, aligned_predictions):
        # Normalize BIO labels (e.g., B-fname → fname, I-fname → fname)
        normalized_slot = slot.replace("b-", "").replace("i-", "")
        if normalized_slot != "o":
            if current_slot == normalized_slot:
                # Append to the current slot value (for multi-token slots)
                slots[normalized_slot] += f" {word}"
            else:
                # Start a new slot
                slots[normalized_slot] = word
                current_slot = normalized_slot

    return slots

# Main function for testing slot filling
def test_slot_filling():
    print("Slot-Filling Model Test")
    print("Type your input (or type 'exit' to quit):")
    
    while True:
        user_input = input("User: ")
        if user_input.lower() == "exit":
            print("Exiting...")
            break

        # Predict slots
        detected_slots = predict_slots(slot_model, slot_tokenizer, user_input, slot_label_encoder)
        
        # Print detected slots
        print("Detected Slots:")
        for slot, value in detected_slots.items():
            print(f"  {slot}: {value}")

# Run the test
if __name__ == "__main__":
    test_slot_filling()

Some weights of XLMRobertaForTokenClassification were not initialized from the model checkpoint at /home/mh/Documents/NLU-exe/xlm_roberta/xlm-roberta-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/tmp/ipykernel_89399/1601704940.py:23: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `t

Slot-Filling Model Test
Type your input (or type 'exit' to quit):
Detected Slots:
  loan_benefit_rate: ۵ درصد
Detected Slots:
  loan_benefit_rate: ۵ درصد
Detected Slots:


In [3]:
import re
from collections import Counter

# Step 1: Load the dataset
def load_dataset(file_path):
    """
    Load the dataset from the given file path.
    Each line contains a sentence and its intent separated by '<=>'.
    """
    with open(file_path, "r", encoding="utf-8") as f:
        lines = f.readlines()
    return lines

# Step 2: Extract intents and slot labels
def extract_intents_and_slots(lines):
    """
    Extract intents and slot labels from the dataset.
    """
    intents = []
    slots_per_intent = {}

    for line in lines:
        # Split the line into the sentence and the intent
        sentence, intent = line.strip().split(" <=> ")
        intents.append(intent)

        # Extract slots (e.g., b-transfer_reason, i-transfer_reason, etc.)
        slots = re.findall(r"\b(b|i)-\w+", sentence)

        # Group slots by intent
        if intent not in slots_per_intent:
            slots_per_intent[intent] = []
        slots_per_intent[intent].extend(slots)

    return intents, slots_per_intent

# Step 3: Count intent frequencies
def count_intent_frequencies(intents):
    """
    Count the frequency of each intent in the dataset.
    """
    intent_counts = Counter(intents)
    return intent_counts

# Step 4: Count specific slot occurrences for a given intent
def count_slot_in_intent(slots_per_intent, target_intent, target_slot):
    """
    Count the occurrences of a specific slot (e.g., b-transfer_reason) in a given intent.
    """
    if target_intent not in slots_per_intent:
        return 0
    return slots_per_intent[target_intent].count(target_slot)

# Main function
def main():
    # Path to the train.txt file
    file_path = "/home/mh/Desktop/NLU-prj/Data-part1&2-v3/train.txt"

    # Load the dataset
    lines = load_dataset(file_path)

    # Extract intents and slots
    intents, slots_per_intent = extract_intents_and_slots(lines)

    # Step 1: Count the frequency of different intents
    intent_counts = count_intent_frequencies(intents)
    print("Intent Frequencies:")
    for intent, count in intent_counts.items():
        print(f"{intent}: {count}")

    # Step 2: Count occurrences of 'b-transfer_reason' in the intent 'card2card'
    target_intent = "card2card"
    target_slot = "b-transfer_reason"
    slot_count = count_slot_in_intent(slots_per_intent, target_intent, target_slot)
    print(f"\nOccurrences of '{target_slot}' in intent '{target_intent}': {slot_count}")

# Run the main function
if __name__ == "__main__":
    main()

Intent Frequencies:
loan_interest: 72
open_account_free: 92
duplicate_card: 119
change_password: 120
delegate_account: 90
software_problem: 110
installment_payment: 90
paya: 92
submit_cheque: 90
signin_problem: 71
card2card: 88
convert_cheque: 88
open_account_current: 79
currency_request: 90
turnover_bill: 90
recieve_cheque: 120
close_card: 120
balance_bill: 90
receipt_payment: 79
open_account_deposit: 74
loan_free: 78

Occurrences of 'b-transfer_reason' in intent 'card2card': 0


In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from sklearn.preprocessing import LabelEncoder

torch.cuda.empty_cache()

# Load intent labels
def load_intent_labels(intent_file_path):
    with open(intent_file_path, "r", encoding="utf-8") as f:
        intent_labels = [line.strip() for line in f.readlines()]
    return intent_labels

# Initialize LabelEncoder for intents
intent_file_path = "/home/mh/Desktop/NLU-prj/Data-part1&2-v3/intent.txt"
intent_labels = load_intent_labels(intent_file_path)

intent_label_encoder = LabelEncoder()
intent_label_encoder.fit(intent_labels)

# Load the intent detection model
intent_model_path = "/home/mh/Downloads/best_augmented_intent_model"
intent_tokenizer = AutoTokenizer.from_pretrained(intent_model_path)
intent_model = AutoModelForSequenceClassification.from_pretrained(intent_model_path)

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
intent_model.to(device)

# Function for intent detection
def predict_intent(text, tokenizer, model, intent_label_encoder):
    """
    Predict the intent of the given text.

    Args:
        text (str): Input text.
        tokenizer: Tokenizer for the model.
        model: Trained intent detection model.
        intent_label_encoder: LabelEncoder for intent labels.

    Returns:
        str: Predicted intent label.
    """
    model.eval()
    encoding = tokenizer(text, padding=True, truncation=True, max_length=128, return_tensors="pt")
    input_ids = encoding["input_ids"].to(device)
    attention_mask = encoding["attention_mask"].to(device)

    with torch.no_grad():
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)

    logits = outputs.logits
    predicted_class = torch.argmax(logits, dim=1).item()
    predicted_intent = intent_label_encoder.inverse_transform([predicted_class])[0]
    return predicted_intent

# Main function to test intent detection
def main():
    print("Intent Detection Test")
    while True:
        user_message = input("Enter your message (or type 'exit' to quit): ")
        if user_message.lower() == "exit":
            print("Exiting...")
            break

        # Predict intent
        intent = predict_intent(user_message, intent_tokenizer, intent_model, intent_label_encoder)
        print(f"Predicted Intent: {intent}")

# Run the intent detection test
if __name__ == "__main__":
    main()